In [1]:
import os

import pandas as pd
from ax import optimize
from omegaconf import DictConfig

from eml.config import Config
from eml.Datasets import load_data
from eml.train import train
import numpy as np


from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting

In [2]:
os.makedirs("hyperopt", exist_ok=True)
os.chdir("hyperopt")


In [3]:

cfg = DictConfig(Config(1, 1))
data_loaders = load_data(cfg)

def evaluation_function(params) -> None:
    cfg.unsupervised_epochs = params["unsupervised_epochs"]
    cfg.classifier_epochs = params["classifier_epochs"]
    cfg.num_train_labels = 60000
    cfg.variational_sigma = params["variational_sigma"]
    cfg.batch_size = 128
    cfg.auto_encoder_channels = [[16, 32, 64], [8, 16, 32], [16, 32]][params["auto_encoder_channels"]]
    cfg.auto_encoder_fc_layers = [2, 4][params["auto_encoder_fc_layers"]]
    cfg.classifier_neurons = [[128, 64, 32], [128, 64], [256, 128, 64], [256, 128, 64, 32]][params["classifier_neurons"]]
    result = train(cfg, *data_loaders)
    return {
        "accuracy": result["classifier/val_acc"]
    }


In [4]:
best_parameters, best_values, experiment, model = optimize(
    parameters=[
        {
            "name": "unsupervised_epochs",
            "type": "range",
            "bounds": [1, 10],
            "value_type": "int",
        },
        {
            "name": "classifier_epochs",
            "type": "range",
            "bounds": [1, 10],
            "value_type": "int",
        },
        {
            "name": "variational_sigma",
            "type": "range",
            "bounds": [0.001, 1.0],
            "value_type": "float",
            "log_scale": True,
        },
        {
            "name": "auto_encoder_channels",
            "type": "choice",
            "values": [0, 1, 2]
        },
        {
            "name": "auto_encoder_fc_layers",
            "type": "range",
            "bounds": [0, 1],
            "value_type": "int",
        },
        {
            "name": "classifier_neurons",
            "type": "choice",
            "values": [0, 1, 2, 3]
        },
    ],
    objective_name="accuracy",
    evaluation_function=evaluation_function,
    minimize=False,
    total_trials=10,
)

[INFO 02-26 00:35:41] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter auto_encoder_channels. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/opt/conda/lib/python3.8/site-packages/ax/core/parameter.py:467: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "auto_encoder_channels". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.

/opt/conda/lib/python3.8/site-packages/ax/core/parameter.py:467: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "auto_encoder_channels". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

[INFO 02-26 00:35:41] ax.service.utils.instantiation: Inferred value type of P

unsupervised_epochs=8 classifier_epochs=6 batch_size=128 auto_encoder_channels=[8, 16, 32] auto_encoder_fc_layers=4 classifier_neurons=[256, 128, 64, 32] variational_sigma=0.2037508531045675
Encoded feature size: 32



  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 10.4 K
1 | decoder | Decoder | 5.9 K 
------------------------------------
16.3 K    Trainable params
0         Non-trainable params
16.3 K    Total params
0.065     Total estimated model params size (MB)


Validation sanity check:  50%|███████████████████████████████████████████████████████████████████                                                                   | 1/2 [00:01<00:01,  1.63s/it]

/opt/conda/lib/python3.8/site-packages/torchvision/utils.py:50: UserWarning:

range will be deprecated, please use value_range instead.


Generating embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 43.74it/s]


...

Epoch 6: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1095/1095 [00:11<00:00, 94.29it/s, loss=0.0305, v_num=7143]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type        | Params
---------------------------------------------
0 | auto_encoder | AutoEncoder | 16.3 K
1 | classifier   | ModuleList  | 15.3 K
---------------------------------------------
31.7 K    Trainable params
0         Non-trainable params
31.7 K    Total params
0.127     Total estimated model params size (MB)
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning:

Checkpoint directory /workspaces/efficient-mnist-learner/hyperopt/default/unsupervised_epochs=7 classifier_epochs=2 batch_size=128 auto_encoder_channels=[8, 16, 32] auto_encoder_fc_layers=4 variational_sigma=0.3051317981697143/checkpoints exists and is not empty.




Epoch 0:  86%|████████████████████████████████████████████████████████████████████████████████████████████▌               | 938/1095 [00:12<00:02, 77.56it/s, loss=0.532, v_num=7143]
Validating: 0it [00:00, ?it/s]
Epoch 1:  86%|████████████████████████████████████████████████████████████████████████████████████████████▌               | 938/1095 [00:11<00:01, 78.83it/s, loss=0.425, v_num=7143]
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1095/1095 [00:12<00:00, 84.65it/s, loss=0.425, v_num=7143]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validating:  89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████              | 140/157 [00:00<00:00, 210.74it/s]--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'classifier/val_acc': 0.8356999754905701,
 'classifier/val_f1': 0.8356999754905701,
 'classifier/val_loss': 0.4625987708568573}
--------------------------------------------------------------------------------
Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 159.73it/s]


In [6]:
#render(plot_contour(model=model, param_x='unsupervised_epochs', param_y='classifier_epochs', metric_name="accuracy"))

In [9]:
best_objectives = np.array([[trial.objective_mean*100 for trial in experiment.trials.values() if trial.index <= 10]])
best_objective_plot = optimization_trace_single_method(
    y=np.maximum.accumulate(best_objectives, axis=1),
    title="Model performance vs. # of iterations",
    ylabel="Classification Accuracy, %",
)
render(best_objective_plot)


In [10]:
list(experiment.trials.values())

[Trial(experiment_name='None', index=0, status=TrialStatus.COMPLETED, arm=Arm(name='0_0', parameters={'unsupervised_epochs': 8, 'classifier_epochs': 6, 'variational_sigma': 0.2037508531045675, 'auto_encoder_channels': 1, 'auto_encoder_fc_layers': 1, 'classifier_neurons': 3})),
 Trial(experiment_name='None', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'unsupervised_epochs': 10, 'classifier_epochs': 5, 'variational_sigma': 0.002523414806095848, 'auto_encoder_channels': 2, 'auto_encoder_fc_layers': 1, 'classifier_neurons': 2})),
 Trial(experiment_name='None', index=2, status=TrialStatus.COMPLETED, arm=Arm(name='2_0', parameters={'unsupervised_epochs': 5, 'classifier_epochs': 10, 'variational_sigma': 0.005069980221161937, 'auto_encoder_channels': 0, 'auto_encoder_fc_layers': 1, 'classifier_neurons': 1})),
 Trial(experiment_name='None', index=3, status=TrialStatus.COMPLETED, arm=Arm(name='3_0', parameters={'unsupervised_epochs': 5, 'classifier_epochs': 1, 'variati

In [11]:
best_parameters

{'unsupervised_epochs': 10,
 'classifier_epochs': 5,
 'variational_sigma': 0.002523414806095848,
 'auto_encoder_channels': 2,
 'auto_encoder_fc_layers': 1,
 'classifier_neurons': 2}

In [ ]:
experiment.fetch_data().df


In [ ]:
experiment